<a href="https://colab.research.google.com/github/humaran27/trabajo_final/blob/main/Trabajo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas gradio openpyxl
import pandas as pd
from openpyxl import Workbook, load_workbook
import os
import gradio as gr

# Cargar el archivo CSV
try:
    df = pd.read_csv('peliculas.csv')
    print("Archivo peliculas.csv cargado correctamente.")
    # Ver la estructura del DataFrame
    print(df.info())
    print(df.head())
except FileNotFoundError:
    print("Error: El archivo 'peliculas.csv' no se encontró. Asegúrate de que el archivo exista en el mismo directorio.")
    df = pd.DataFrame({'Título': [], 'Género': [], 'Comentarios': []}) # Crear un DataFrame vacío
except Exception as e:
    print(f"Error al cargar el archivo CSV: {e}")
    df = pd.DataFrame({'Título': [], 'Género': [], 'Comentarios': []})

def buscar_pelicula(pregunta):
    # Interpretamos la pregunta y buscamos en la base de datos
    pregunta_lower = pregunta.lower()
    if 'género' in pregunta_lower:
        genero = pregunta_lower.split('género')[-1].strip()
        resultado = df[df['Género'].str.contains(genero, case=False, na=False)]
        return resultado[['Título', 'Género', 'Comentarios']]

    elif 'nombre' in pregunta_lower:
        nombre = pregunta_lower.split('nombre')[-1].strip()
        resultado = df[df['Título'].str.contains(nombre, case=False, na=False)]
        return resultado[['Título', 'Género', 'Comentarios']]

    else:
        return "No pude entender la pregunta. Intenta con 'género' o 'nombre'."

def calificar_comentarios(comentarios):
    if isinstance(comentarios, str):
        comentarios = comentarios.lower()
        palabras_buenas = ["excelente", "buena", "genial", "fantástica", "increíble", "maravillosa", "disfruté", "recomendable"]
        palabras_malas = ["mala", "horrible", "aburrida", "predecible", "débil", "decepcionante", "pérdida de tiempo"]
        for palabra in palabras_buenas:
            if palabra in comentarios:
                return "Buena"
        for palabra in palabras_malas:
            if palabra in comentarios:
                return "Mala"
    return "Neutral"

# Crear un archivo Excel si no existe
if not os.path.exists('preguntas.xlsx'):
    wb = Workbook()
    ws = wb.active
    ws.append(['Pregunta'])
    wb.save('preguntas.xlsx')

def guardar_pregunta(pregunta):
    try:
        wb = load_workbook('preguntas.xlsx')
        ws = wb.active
        ws.append([pregunta])
        wb.save('preguntas.xlsx')
    except FileNotFoundError:
        wb = Workbook()
        ws = wb.active
        ws.append(['Pregunta'])
        wb.save('preguntas.xlsx')
    except Exception as e:
        print(f"Error al guardar la pregunta: {e}")

# Función que combina todo
def interfaz(pregunta):
    # Guardamos la pregunta
    guardar_pregunta(pregunta)

    # Buscamos la película basada en la pregunta
    resultado_busqueda = buscar_pelicula(pregunta)

    # Si no hay resultados, mostramos un mensaje adecuado
    if isinstance(resultado_busqueda, str):
        return resultado_busqueda
    elif resultado_busqueda.empty:
        return "No se encontraron películas con los criterios especificados."
    else:
        # Ahora procesamos la calificación para cada comentario
        resultado_busqueda.loc[:, 'Calificación'] = resultado_busqueda['Comentarios'].apply(calificar_comentarios)
        return resultado_busqueda[['Título', 'Género', 'Comentarios', 'Calificación']]

# Crear la interfaz de Gradio
with gr.Blocks(theme=gr.themes.Soft()) as iface:
    gr.Markdown("# Buscador y Calificador de Películas")
    with gr.Row():
        pregunta_input = gr.Textbox(label="Ingresa tu pregunta (por género o nombre de película):")
    buscar_button = gr.Button("Buscar Película")
    resultado_output = gr.DataFrame(label="Resultados de la búsqueda")
    error_output = gr.Textbox(label="Mensaje de Error", visible=False, elem_id="error_output") # Asignamos un elem_id

    def buscar_y_manejar_errores(pregunta):
        try:
            resultado = interfaz(pregunta)
            return resultado, "" # Devolvemos el resultado y un string vacío para el error
        except Exception as e:
            print(f"Ocurrió un error durante la búsqueda: {e}")
            return None, str(e) # Devolvemos None para el resultado y el mensaje de error

    buscar_button.click(fn=buscar_y_manejar_errores,
                        inputs=pregunta_input,
                        outputs=[resultado_output, error_output])

    def mostrar_error(error_mensaje):
        return gr.Textbox(visible=bool(error_mensaje), value=error_mensaje)

    #iface.load(inputs=error_output, outputs=error_output, fn=mostrar_error, every=1)

    gr.Markdown("---")
    gr.Markdown("### Historial de Preguntas:")
    historial_output = gr.File(label="Descargar Historial de Preguntas (preguntas.xlsx)")

    def actualizar_historial():
        if os.path.exists('preguntas.xlsx'):
            return 'preguntas.xlsx'
        else:
            return None

    # iface.load(inputs=None, outputs=historial_output, fn=actualizar_historial, every=5)

# Lanzar la interfaz
iface.launch()